<a href="https://colab.research.google.com/github/xuxiufeng/Spam_ham_Classification/blob/main/Text%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Unzip files

In [ ]:
%%capture
!unzip enron1_test.zip
!unzip enron1_train.zip
!unzip enron4_test.zip
!unzip enron4_train.zip
!unzip hw1.zip # upload hw(including train and set) 1 to colab


## Stop Words

step words from the nltk library
              

In [177]:
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 
              "you're", "you've","you'll", "you'd", 'your', 'yours', 'yourself', 
              'yourselves', 'he', 'him', 'his','himself', 'she', "she's", 'her', 
              'hers', 'herself', 'it', "it's", 'its', 'itself','they', 'them', 
              'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom',
              'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 
              'was', 'were', 'be','been', 'being', 'have', 'has', 'had', 'having', 
              'do', 'does', 'did', 'doing', 'a','an', 'the', 'and', 'but', 'if', 
              'or', 'because', 'as', 'until', 'while', 'of', 'at','by', 'for', 
              'with', 'about', 'against', 'between', 'into', 'through', 'during',
              'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 
              'out', 'on','off', 'over', 'under', 'again', 'further', 'then', 'once', 
              'here', 'there', 'when','where', 'why', 'how', 'all', 'any', 'both', 
              'each', 'few', 'more', 'most', 'other','some', 'such', 'no', 'nor', 'not', 
              'only', 'own', 'same', 'so', 'than', 'too', 'very','s', 't', 'can', 'will', 
              'just', 'don', "don't", 'should', "should've", 'now', 'd','ll', 'm', 'o', 're', 
              've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 
              'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't",
              'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't",
              'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won',
              "won't", 'wouldn', "wouldn't"]

## Library

In [178]:
import copy
import glob
import os
import re
import random
import codecs
import numpy as np
from decimal import Decimal
from math import log
from collections import Counter
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

## Import Data

*  path_of_dataset_name: The path of folder. For example, 'content/enron1'
*  True_or_False_train_set: True means it is train set; otherwise, it is test set






In [179]:
def import_data_train_or_test(path_of_dataset_name, True_or_False_train_set):
    all_ham_emails_raw_content = []
    all_spam_emails_raw_content = []
    all_spam_ham_content = ""
    if True_or_False_train_set == True:
        path = path_of_dataset_name + '/train'
    else:
        path = path_of_dataset_name + '/test'
    path_ham = path + '/ham/'
    path_spam = path + '/spam/'
    all_spam_emails_raw_contents_list = os.listdir(path_spam)
    all_ham_emails_raw_contents_list = os.listdir(path_ham)
    for all_spam_emails_raw_contents in all_spam_emails_raw_contents_list:
      total_spam_path = path_spam + all_spam_emails_raw_contents
      all_spam_emails_raw_content.append(open(total_spam_path, "r", errors = 'ignore').read())
      all_spam_ham_content = all_spam_ham_content + " " + open(total_spam_path, "r", errors = 'ignore').read()
    for all_ham_emails_raw_contents in all_ham_emails_raw_contents_list:
      total_ham_path = path_ham + all_ham_emails_raw_contents
      all_ham_emails_raw_content.append(open(total_ham_path, "r", errors = 'ignore').read())
      all_spam_ham_content = all_spam_ham_content + " " + open(total_ham_path, "r", errors = 'ignore').read()
    # we find the size of the dataset and the number of instances with spam and number of instances with ham
    num_of_files_whole_dataset = len(all_ham_emails_raw_contents_list) + len(all_spam_emails_raw_contents_list)
    num_of_files_ham_dataset = len(all_ham_emails_raw_contents_list)
    num_of_files_spam_dataset = len(all_spam_emails_raw_contents_list)
    return all_spam_emails_raw_content, all_ham_emails_raw_content, all_spam_ham_content, num_of_files_whole_dataset, num_of_files_spam_dataset, num_of_files_ham_dataset

## Bag of Words model

*  path_of_dataset_name: The path of folder. For example, 'content/enron1'
*  True_or_False_train_set: True means it is train set; otherwise, it is test set

In [180]:
def bag_of_words_model(path_of_dataset_name, True_or_False_train_set):
    all_spam_emails_raw_content, all_ham_emails_raw_content, all_spam_ham_content, num_of_files_whole_dataset, num_of_files_spam_dataset, num_of_files_ham_dataset = import_data_train_or_test(path_of_dataset_name, True_or_False_train_set)
    total_words_dictionary = {} # total words dictionary
    total_file_words = re.findall("[a-zA-Z]+", all_spam_ham_content)
    words_frequency_in_all_folders = {}
    for each_word in total_file_words:
        each_word = each_word.lower()
              # get dictionary of each word
        if each_word in total_words_dictionary:
            continue
        else:    
            if each_word not in stop_words:
                total_words_dictionary[each_word] = 0
                # get frequency of each word
        if each_word in words_frequency_in_all_folders:
            if each_word not in stop_words:
                words_frequency_in_all_folders[each_word] = words_frequency_in_all_folders[each_word] + 1
        else:
            if each_word not in stop_words:
                words_frequency_in_all_folders[each_word] = 1
    # bag of words for spam emails
    spam_email_bag_of_words = []
    words_frenquency_spam_email_all_folders = {}
    for each_spam_email in all_spam_emails_raw_content:
        temporary_dictionary = copy.deepcopy(total_words_dictionary)
        words_in_each_spam_email = re.findall("[a-zA-Z]+", each_spam_email) # only words in alphabets
        for each_word in words_in_each_spam_email:
            each_word = each_word.lower()
            if each_word in temporary_dictionary:
                temporary_dictionary[each_word] = temporary_dictionary[each_word] + 1
        # Here we store all the words in the spam dataset
        words_frenquency_spam_email_all_folders = Counter(words_frenquency_spam_email_all_folders) + Counter(temporary_dictionary) # total frenquency for spam emails
        spam_email_bag_of_words.append(temporary_dictionary) # give frenquency of words in each spam email
    # bag of words for ham emails
    words_frenquency_ham_email_in_all_folders = {}
    ham_email_bag_of_words = []
    for each_ham_mail in all_ham_emails_raw_content:
        temporary_dictionary = copy.deepcopy(total_words_dictionary)
        words_in_each_ham_email = re.findall("[a-zA-Z]+", each_ham_mail)
        for each_word in words_in_each_ham_email:
            each_word = each_word.lower()
            if each_word in temporary_dictionary:
                temporary_dictionary[each_word] = temporary_dictionary[each_word] + 1
        # Here we store all the words in the ham dataset
        words_frenquency_ham_email_in_all_folders = Counter(words_frenquency_ham_email_in_all_folders) + Counter(temporary_dictionary) # total frequency for ham emails
        ham_email_bag_of_words.append(temporary_dictionary) # give frenquency of words in each ham email
    return spam_email_bag_of_words, ham_email_bag_of_words, words_frequency_in_all_folders, words_frenquency_spam_email_all_folders, words_frenquency_ham_email_in_all_folders, num_of_files_whole_dataset, num_of_files_spam_dataset, num_of_files_ham_dataset, total_words_dictionary

## Bernoulli model

*  path_of_dataset_name: The path of folder. For example, 'content/enron1'
*  True_or_False_train_set: True means it is train set; otherwise, it is test set

In [181]:
def bernoulli_model(path_of_dataset_name, True_or_False_train_set):
    all_spam_emails_raw_content, all_ham_emails_raw_content, all_spam_ham_content, num_of_files_whole_dataset, num_of_files_spam_dataset, num_of_files_ham_dataset = import_data_train_or_test(path_of_dataset_name, True_or_False_train_set)
    total_words_dictionary = {}
    total_file_words = re.findall("[a-zA-Z]+", all_spam_ham_content)
    # at first we find all the words in the given dataset and find the occurrences in the whole dataset
    for each_word in total_file_words:   
        each_word = each_word.lower() # lower case form
        if each_word in total_words_dictionary:
            continue
        else:
            if each_word not in stop_words:
                total_words_dictionary[each_word] = 0
    # bernoulli model for spam
    spam_email_bernoulli_model = []
    words_occurence_spam_email_all_folders = {} #
    for each_spam_email in all_spam_emails_raw_content:
        
        temporary_dictionary = copy.deepcopy(total_words_dictionary)
        words_in_each_spam_email = re.findall("[a-zA-Z]+", each_spam_email)
        for each_word in words_in_each_spam_email:
            each_word = each_word.lower() # lower case form
            if each_word in temporary_dictionary:
                temporary_dictionary[each_word] = 1           
                words_occurence_spam_email_all_folders[each_word] = 1 
        spam_email_bernoulli_model.append(temporary_dictionary)
    # bernoulli model for ham
    ham_email_bernoulli_model = []
    words_occurence_ham_email_in_all_folders = {}
    for each_ham_mail in all_ham_emails_raw_content:
        
        temporary_dictionary = copy.deepcopy(total_words_dictionary)
        words_in_each_ham_email = re.findall("[a-zA-Z]+", each_ham_mail)
        for each_word in words_in_each_ham_email:
            each_word = each_word.lower()
            if each_word in temporary_dictionary:
                temporary_dictionary[each_word] = 1
                words_occurence_ham_email_in_all_folders[each_word] = 1
        ham_email_bernoulli_model.append(temporary_dictionary)
    return spam_email_bernoulli_model, ham_email_bernoulli_model, words_occurence_spam_email_all_folders, words_occurence_ham_email_in_all_folders, num_of_files_whole_dataset, num_of_files_spam_dataset, num_of_files_ham_dataset, total_words_dictionary

## Evaluation Function

In [182]:
def accuracy(real_y, predicted_y):
    correct_predictions = 0
    for each in range(len(real_y)):
        if real_y[each] == predicted_y[each]:
            correct_predictions += 1
    return correct_predictions / len(real_y)


def precision(real_y, predicted_y):
    true_positives = 0
    false_positives = 0
    for each in range(len(real_y)):
        if real_y[each] == predicted_y[each] and predicted_y[each] == 1:
            true_positives += 1
        if real_y[each] != predicted_y[each] and predicted_y[each] == 1:
            false_positives += 1
    return true_positives / (true_positives + false_positives)


def recall(real_y, predicted_y):
    true_positives = 0
    false_negetives = 0
    for each in range(len(real_y)):
        if real_y[each] == predicted_y[each] and predicted_y[each] == 1:
            true_positives += 1
        if real_y[each] != predicted_y[each] and predicted_y[each] == 0:
            false_negetives += 1
    return true_positives / (true_positives + false_negetives)


def f1_score(recall, precision):
    return (2 * recall * precision) / (recall + precision)

## multinomial Naive Bayes

* total_words_dictionary: dictionary of all of words in dataset
* spam_email_bag_of_words:  frequence of words in each spam email(from the first to the last email)
* ham_email_bag_of_words: frequence of words in each ham email(from the first to the last email)
* words_frequency_in_all_folders: total words frequencies
* words_frenquency_spam_email_all_folders: total frequencies in all spam emails
* words_frenquency_ham_email_in_all_folders: total frequencies in all ham emails
* num_of_files_whole_dataset: the number of files of whole dataset
* num_of_files_spam_dataset: the number of files of spam dataset
* num_of_files_ham_dataset: the number of files of ham dataset

* log_prior: $log(P(Y=c_i))$
* log_conditional_probability: $[log(p_{ij})]$, which is a vector


In [183]:
def train_multinomial_naive_bayes(spam_email_bag_of_words, ham_email_bag_of_words, words_frequency_in_all_folders,
                         words_frenquency_spam_email_all_folders, words_frenquency_ham_email_in_all_folders, num_of_files_whole_dataset,
                         num_of_files_spam_dataset, num_of_files_ham_dataset, total_words_dictionary):
    # variables to store the values
    log_prior = {}
    log_conditional_probability = {}
    log_conditional_probability["spam"] = {}
    log_conditional_probability["ham"] = {}
    log_conditional_probability_of_non_occurring_word = {}
    log_conditional_probability_of_non_occurring_word["spam"] = {}
    log_conditional_probability_of_non_occurring_word["ham"] = {}
    
    # log_priors for the spam and ham dataset
    log_prior["spam"] = log(num_of_files_spam_dataset / float(num_of_files_whole_dataset)) # log-scale to avoid underflow
    total_number_of_words_in_ham = sum(words_frenquency_ham_email_in_all_folders.values())
    log_prior["ham"] = log(num_of_files_ham_dataset / float(num_of_files_whole_dataset))
    total_number_of_words_in_spam = sum(words_frenquency_ham_email_in_all_folders.values())
    # Now we calculate the values for the conditional probabilities
    for each_word in list(words_frenquency_spam_email_all_folders):
        log_conditional_probability["spam"][each_word] = log((words_frenquency_spam_email_all_folders[each_word] + 1) / (
            float(total_number_of_words_in_spam + len(words_frequency_in_all_folders))))

    # same procedure for ham docs
    for each_word in list(words_frenquency_ham_email_in_all_folders):
        log_conditional_probability["ham"][each_word] = log((words_frenquency_ham_email_in_all_folders[each_word] + 1) / (
            float(total_number_of_words_in_ham + len(words_frequency_in_all_folders))))
    # log conditional probabilities not in the training dataset
    log_conditional_probability_of_non_occurring_word["ham"] = log(
        1 / (float(total_number_of_words_in_ham + len(words_frequency_in_all_folders))))
    log_conditional_probability_of_non_occurring_word["spam"] = log(
        1 / (float(total_number_of_words_in_spam + len(words_frequency_in_all_folders))))
    return log_prior, log_conditional_probability, log_conditional_probability_of_non_occurring_word

* log_conditional_probability_of_non_occurring_word: log conditional probability for each word in the testing set which is not in the training data
* an_email_bag_of_words_test: an email in test dataset we want to classify
* 1: spam 0: ham

In [184]:
def test_multinomial_naive_bayes(log_prior, log_conditional_probability, log_conditional_probability_of_non_occurring_word,
                                 an_email_bag_of_words_test):
    weight_of_class = {}
    for each_class in list(log_prior):
        weight_of_class[each_class] = log_prior[each_class]
        for each_word in list(an_email_bag_of_words_test):
            if an_email_bag_of_words_test[each_word] != 0:
                try:
                    weight_of_class[each_class] += log_conditional_probability[each_class][each_word]
                # This is the case if the word was not in the train data and thus the laplace pruning gives this result
                except KeyError:
                    weight_of_class[each_class] += log_conditional_probability_of_non_occurring_word[each_class]
    # Here we are taking spam as 1 and ham as -1
    if weight_of_class["spam"] > weight_of_class["ham"]:
        return 1
    else:
        return 0

In [185]:
def evaluate_multinomial_naive_bayes(path_of_dataset_name):
    """
    This is the method used for evaluation of multinomial NB on a particular dataset
    :param path_of_dataset_name: This is the given dataset name
    :return: The method returns the accuracy, precision, recall and f1_score for the given dataset
    """
    # We first import training data for the training
    try:
        spam_email_bag_of_words, ham_email_bag_of_words, words_frequency_in_all_folders, words_frenquency_spam_email_all_folders, words_frenquency_ham_email_in_all_folders, num_of_files_whole_dataset, num_of_files_spam_dataset, num_of_files_ham_dataset, total_words_dictionary = bag_of_words_model(
            path_of_dataset_name, True)
    except:
        print ("wrong file")
        exit(-1)
    log_prior, log_conditional_probability, log_conditional_probability_of_non_occurring_word = train_multinomial_naive_bayes(
        spam_email_bag_of_words, ham_email_bag_of_words, words_frequency_in_all_folders, words_frenquency_spam_email_all_folders,
        words_frenquency_ham_email_in_all_folders, num_of_files_whole_dataset, num_of_files_spam_dataset, num_of_files_ham_dataset,
        total_words_dictionary)
    # We now import the data for testing
    spam_email_bag_of_words, ham_email_bag_of_words, words_frequency_in_all_folders, words_frenquency_spam_email_all_folders, words_frenquency_ham_email_in_all_folders, num_of_files_whole_dataset, num_of_files_spam_dataset, num_of_files_ham_dataset, total_words_dictionary = bag_of_words_model(
        path_of_dataset_name, False)
    # We calculate the evaluation metric
    # Here we first predict for the spam class and then the ham class
    spam_predict = []
    for each_document in spam_email_bag_of_words:
        spam_predict.append(test_multinomial_naive_bayes(log_prior, log_conditional_probability,
                                                                                  log_conditional_probability_of_non_occurring_word,
                                                                                  each_document))
    # We  are taking spam as 1
    spam_actual = [1] * len(spam_predict)
    ham_predict = []
    for each_document in ham_email_bag_of_words:
        ham_predict.append(test_multinomial_naive_bayes(log_prior, log_conditional_probability,
                                                                                 log_conditional_probability_of_non_occurring_word,
                                                                                 each_document))
    ham_actual = [0] * len(ham_predict)
    total_actual = spam_actual + ham_actual
    total_predict = spam_predict + ham_predict
    # Now we find the evaluation metrics for the method
    accur = accuracy(total_actual, total_predict)
    prec = precision(total_actual, total_predict)
    rec = recall(total_actual, total_predict)
    f1 = f1_score(rec, prec)
    return accur, prec, rec, f1

accuracy, precision, recall, and F1 score

In [186]:
evaluate_multinomial_naive_bayes('/content/enron1')

(0.8114035087719298, 1.0, 0.4228187919463087, 0.5943396226415094)

In [187]:
evaluate_multinomial_naive_bayes('/content/enron4')

(0.8913443830570903, 0.8688888888888889, 1.0, 0.929845422116528)

In [188]:
evaluate_multinomial_naive_bayes('/content/hw1')

(0.8326359832635983, 1.0, 0.38461538461538464, 0.5555555555555556)

## discrete Naive Bayes

* total_words_dictionary: total words dictionary
* words_occurence_spam_email_all_folders: bernoulli model of all the spam folders
* words_occurence_ham_email_in_all_folders: bernoulli model of all the spam folders

In [189]:
def train_discrete_naive_bayes(spam_email_bernoulli_model, ham_email_bernoulli_model,
                               words_occurence_spam_email_all_folders, words_occurence_ham_email_in_all_folders, num_of_files_whole_dataset,
                               num_of_files_spam_dataset, num_of_files_ham_dataset, total_words_dictionary):
    log_prior = {}
    # variables to store the values
    log_conditional_probability = {}
    log_conditional_probability["spam"] = {}
    log_conditional_probability["ham"] = {}
    log_conditional_probability_of_non_occurring_word = {}
    log_conditional_probability_of_non_occurring_word["spam"] = {}
    log_conditional_probability_of_non_occurring_word["ham"] = {}
    # log_prior of both the classes
    log_prior["spam"] = log(num_of_files_spam_dataset / num_of_files_whole_dataset)
    log_prior["ham"] = log(num_of_files_ham_dataset / num_of_files_whole_dataset)
    # We are doing 1-laplace smoothing and thus we add 1 in the numerator and 2 in denominator(since each word can
    # have two values 0, 1 )
    sum_of_words_occurence_each_spam_email_all_folders = {}
    for ind in range(len(spam_email_bernoulli_model)):
      sum_of_words_occurence_each_spam_email_all_folders = Counter(sum_of_words_occurence_each_spam_email_all_folders)+Counter(spam_email_bernoulli_model[ind])

    sum_of_words_occurence_each_ham_email_all_folders = {}
    for ind in range(len(ham_email_bernoulli_model)):
      sum_of_words_occurence_each_ham_email_all_folders = Counter(sum_of_words_occurence_each_ham_email_all_folders)+Counter(ham_email_bernoulli_model[ind])

    for each_word in words_occurence_spam_email_all_folders:     
        log_conditional_probability["spam"][each_word] = log(
            1 + sum_of_words_occurence_each_spam_email_all_folders[each_word] / (num_of_files_spam_dataset + 2)) # num_of_files_spam_dataset represents the number of spam emails

    for each_word in words_occurence_ham_email_in_all_folders:
        log_conditional_probability["ham"][each_word] = log(
            1 + sum_of_words_occurence_each_ham_email_all_folders[each_word] / num_of_files_ham_dataset + 2)
    # log probabilities for the word not in the training data and appear in the testing data
    log_conditional_probability_of_non_occurring_word["ham"] = log(1 / (num_of_files_ham_dataset + 2))
    log_conditional_probability_of_non_occurring_word["spam"] = log(1 / (num_of_files_spam_dataset + 2))
    return log_prior, log_conditional_probability, log_conditional_probability_of_non_occurring_word

1: spam 0: ham

In [190]:
def test_discrete_naive_bayes(log_prior, log_conditional_probability, log_conditional_probability_of_non_occurring_word,
                              an_email_bag_of_words_test):
    weight_of_class = {}
    # find the words in the given documents for each class and find the numerator of posterior
    for each_class in list(log_prior):
        weight_of_class[each_class] = log_prior[each_class]
        for each_word in list(an_email_bag_of_words_test):
            if an_email_bag_of_words_test[each_word] != 0:
                try:
                    weight_of_class[each_class] += log_conditional_probability[each_class][each_word]
                # This is the case if the word was not in the train data and thus the laplace pruning gives this result
                except KeyError:
                    weight_of_class[each_class] += log_conditional_probability_of_non_occurring_word[each_class]
    if weight_of_class["spam"] > weight_of_class["ham"]:
        return 1
    else:
        return 0

In [191]:
def evaluate_discrete_naive_bayes(path_of_dataset_name):
    # import training data for training
    try:
        spam_email_bernoulli_model, ham_email_bernoulli_model, words_occurence_spam_email_all_folders, words_occurence_ham_email_in_all_folders, num_of_files_whole_dataset, num_of_files_spam_dataset, num_of_files_ham_dataset, total_words_dictionary = bernoulli_model(
            path_of_dataset_name, True)
    except:
        print ("wrong file")
        exit(-1)
    log_prior, log_conditional_probability, log_conditional_probability_of_non_occurring_word = train_discrete_naive_bayes(
        spam_email_bernoulli_model, ham_email_bernoulli_model, words_occurence_spam_email_all_folders, words_occurence_ham_email_in_all_folders, num_of_files_whole_dataset, num_of_files_spam_dataset, num_of_files_ham_dataset,
        total_words_dictionary)
    # import the data for testing
    spam_email_bernoulli_model, ham_email_bernoulli_model, words_occurence_spam_email_all_folders, words_occurence_ham_email_in_all_folders, num_of_files_whole_dataset, num_of_files_spam_dataset, num_of_files_ham_dataset, total_words_dictionary = bernoulli_model(
        path_of_dataset_name, False)
    # We calculate the evaluation metric
    # Here we first predict for the spam class and then the ham class
    spam_predict = []
    for each_document in spam_email_bernoulli_model:
        spam_predict.append(test_discrete_naive_bayes(log_prior, log_conditional_probability,
                                                                           log_conditional_probability_of_non_occurring_word,
                                                                           each_document))
    # We  are taking spam as 1
    spam_actual = [1] * len(spam_predict)
    ham_predict = []
    for each_document in ham_email_bernoulli_model:
        ham_predict.append(test_discrete_naive_bayes(log_prior, log_conditional_probability,
                                                                          log_conditional_probability_of_non_occurring_word,
                                                                          each_document))
    ham_actual = [0] * len(ham_predict)
    total_actual = spam_actual + ham_actual
    total_predict = spam_predict + ham_predict
    # evaluation
    accur = accuracy(total_actual, total_predict)
    prec = precision(total_actual, total_predict)
    rec = recall(total_actual, total_predict)
    f1 = f1_score(rec, prec)
    return accur, prec, rec, f1

In [192]:
evaluate_discrete_naive_bayes('/content/enron1')

(0.8947368421052632,
 0.9902912621359223,
 0.6845637583892618,
 0.8095238095238094)

In [193]:
evaluate_discrete_naive_bayes('/content/enron4')

(0.9060773480662984, 0.967032967032967, 0.9002557544757033, 0.9324503311258278)

In [194]:
evaluate_discrete_naive_bayes('/content/hw1')

(0.9163179916317992, 1.0, 0.6923076923076923, 0.8181818181818181)

## MCAP Logistic Regression



*   spam_email_model: spam_email_bag_of_words or spam_email_bernoulli_model
*   ham_email_model: ham_mail_bag_of_words or ham_email_bernoulli_model



In [195]:
def divide_into_validation_and_train(spam_email_model, ham_email_model):
    # Here spam is 1 and ham is 0 (since we are using sigmoid)
    for ind in range(len(spam_email_model)):
        spam_email_model[ind]["the_class_of_the_document"] = 1
        spam_email_model[ind]["zero_weight"] = 1
    for ind in range(len(ham_email_model)):
        ham_email_model[ind]["the_class_of_the_document"] = 0
        ham_email_model[ind]["zero_weight"] = 1
    all_data = spam_email_model + ham_email_model
    # We are using this step to shuffle our data so that different data goes into training and testing everything
    random.shuffle(all_data)
    # 70 percent of the data for traning and 30 percent of the data for validation
    train_data = all_data[0: int(len(all_data) * .70)]
    validation_data = all_data[int(len(all_data) * .70): -1]
    return train_data, validation_data

value: $w_0+\sum_{i=1}^nw_iX_i$

In [196]:
def get_output_for_class(weights, inputs):
    value = weights['zero_weight'] * 1
    for each in inputs:
        if each == 'the_class_of_the_document' or each == 'zero_weight':
            continue
        else:
            if each in weights and each in inputs:
                value = value + (weights[each] * inputs[each])
    return value

$P(Y=1|X)=\frac{1}{1+exp(-(w_0+\sum_{i=1}^nw_iX_i))}$

In [197]:
def get_posterior(weights, inputs):
    value = weights['zero_weight'] * 1
    for each in inputs:
        if each == 'the_class_of_the_document' or each == 'zero_weight':
            continue
        else:
            if each in weights and each in inputs:
                value = value + (weights[each] * inputs[each])
    return 1 / (1 + np.exp(-value))


This function return the optimized weights
*   $W_0 ← w_0+\eta \sum_l (Y^l-\hat{P}(Y^l=1\mid X^l,W))-\eta \lambda w_0$
*   $W_i ← w_i+\eta \sum_l X_i^l(Y^l-\hat{P}(Y^l=1\mid X^l,W))-\eta \lambda w_i$
*   eta: $\eta$ - learning rate
*   lambda_parameter: $\lambda$



In [198]:
def train_mcap_logistic_regression(train_data, total_words_dictionary, eta, lambda_parameter, number_of_iterations):
    # w_0 outside the array
    weights = copy.deepcopy(total_words_dictionary)
    for each in weights:
        weights[each] = 0
    weights['zero_weight'] = 0
    # update all the weights
    for each in range(number_of_iterations):
        for each_instance in train_data:
            posterior = get_posterior(weights, each_instance)
            sum_of_vals = 0
            for each_weight in weights:
                # if the weight is not equal to 0 or not
                if each_instance[each_weight] != 0:
                    if each_weight == "zero_weight":
                        sum_of_vals = sum_of_vals + eta * (
                                each_instance["the_class_of_the_document"] - posterior)
                    else:
                        sum_of_vals = sum_of_vals + eta * (each_instance[each_weight] * (
                                each_instance["the_class_of_the_document"] - posterior))
                    weights[each_weight] = weights[each_weight] + sum_of_vals - eta * lambda_parameter * weights[
                        each_weight]
    return weights

In [199]:
def test_mcap_logistic_regression(test_example, weights):
    value = get_output_for_class(weights, test_example)
    # 0 is ham and 1 is spam
    if value < 0:
        return 0
    else:
        return 1

Do grid search to find $\lambda$ from $1$ to $8$ increasing $2$ at a time.

In [200]:
def mcap_validation(train_data, validation_data, total_words_dictionary):
    
    eta = 0.01
    max_accuracy = 0
    best_lambda_value = 2
    # We take the range from 1 increasing 2 at a time
    for each_lambda_value in range(1, 8, 2):
        # training model using training dataset
        weights = train_mcap_logistic_regression(train_data, total_words_dictionary, eta, each_lambda_value, 50)
        correct_classification = 0
        # We test on the validation data
        for each_document in validation_data:
            output = test_mcap_logistic_regression(each_document, weights)
            if output == each_document["the_class_of_the_document"]:
                correct_classification += 1
        accuracy = correct_classification / len(validation_data)
        # get the best lambda value
        if accuracy > max_accuracy:
            max_accuracy = accuracy
            best_lambda_value = each_lambda_value
    return best_lambda_value

In [201]:
def evaluate_MCAP_bag_of_words(path_of_dataset_name):
    # import training data for the training
    try:
        spam_email_bag_of_words, ham_email_bag_of_words, words_frequency_in_all_folders, words_frenquency_spam_email_all_folders, words_frenquency_ham_email_in_all_folders, num_of_files_whole_dataset, num_of_files_spam_dataset, num_of_files_ham_dataset, total_words_dictionary = bag_of_words_model(
            path_of_dataset_name, True)
    except:
        print ("wrong file")
        exit(-1)
    # divide training data into training and validation data
    train_data, validation_data = divide_into_validation_and_train(spam_email_bag_of_words,
                                                                                            ham_email_bag_of_words)
    # get the lambda by using the grid search algorithm
    lambda_parameter = mcap_validation(train_data, validation_data, total_words_dictionary)
    # merge the training data and the validation data
    train_data = train_data + validation_data
    eta = 0.01
    # In this step the algorithm learns the weights
    weights = train_mcap_logistic_regression(train_data, total_words_dictionary, eta,
                                                                      lambda_parameter, 100)
    # import testing data
    spam_email_bag_of_words_test, ham_email_bag_of_words_test, words_frequency_in_all_folders_test, words_frenquency_spam_email_all_folders_test, words_frenquency_ham_email_in_all_folders_test, num_of_files_whole_dataset_test, num_of_files_spam_dataset_test, num_of_files_ham_dataset_test, total_words_dictionary_test = bag_of_words_model(
        path_of_dataset_name, False)
    spam_predict = []
    # do prediction given test dataset
    for each_document in spam_email_bag_of_words_test:
        spam_predict.append(test_mcap_logistic_regression(each_document, weights))
    # spam: 1 ham: 0
    spam_actual = [1] * len(spam_predict)
    ham_predict = []
    for each_document in ham_email_bag_of_words_test:
        ham_predict.append(test_mcap_logistic_regression(each_document, weights))
    ham_actual = [0] * len(ham_predict)
    total_actual = spam_actual + ham_actual
    total_predict = spam_predict + ham_predict
    # Now we find the evaluation metrics for the method
    accur = accuracy(total_actual, total_predict)
    prec = precision(total_actual, total_predict)
    rec = recall(total_actual, total_predict)
    f1 = f1_score(rec, prec)
    return accur, prec, rec, f1, lambda_parameter
    

In [202]:
evaluate_MCAP_bag_of_words('/content/enron1')

(0.9122807017543859,
 0.9658119658119658,
 0.7583892617449665,
 0.849624060150376,
 3)

In [203]:
evaluate_MCAP_bag_of_words('/content/enron4')

(0.9613259668508287, 0.9490291262135923, 1.0, 0.9738480697384807, 1)

In [204]:
evaluate_MCAP_bag_of_words('/content/hw1')

(0.9246861924686193,
 0.9519230769230769,
 0.7615384615384615,
 0.8461538461538461,
 5)

In [205]:
def evaluate_MCAP_bernoulli_model(path_of_dataset_name):
    # import training data for the training
    try:
        spam_email_bernoulli_model1, ham_email_bernoulli_model1, words_frenquency_spam_email_all_folders, words_frenquency_ham_email_in_all_folders, num_of_files_whole_dataset, num_of_files_spam_dataset, num_of_files_ham_dataset, total_words_dictionary = bernoulli_model(
            path_of_dataset_name, True)
    except:
        print ("wrong file")
        exit(-1)
    # divide training data into training and validation data
    train_data, validation_data = divide_into_validation_and_train(spam_email_bernoulli_model1,
                                                                                            ham_email_bernoulli_model1)
    # get the lambda value by using the grid search algorithm
    lambda_parameter = mcap_validation(train_data, validation_data, total_words_dictionary)
    eta = 0.01
    # merge the training data and the validation data again
    train_data = train_data + validation_data
    # In this step the algorithm learns the weights
    weights = train_mcap_logistic_regression(train_data, total_words_dictionary, eta,
                                                                      lambda_parameter, 100)
    # import testing data
    spam_email_bernoulli_model_test, ham_email_bernoulli_model_test, words_frenquency_spam_email_all_folders_test, words_frenquency_ham_email_in_all_folders_test, num_of_files_whole_dataset_test, num_of_files_spam_dataset_test, num_of_files_ham_dataset_test, total_words_dictionary_test = bernoulli_model(
        path_of_dataset_name, False)
    spam_predict = []
    # do prediction given test dataset
    for each_document in spam_email_bernoulli_model_test:
        spam_predict.append(test_mcap_logistic_regression(each_document, weights))
    # spam: 1 ham: 0
    spam_actual = [1] * len(spam_predict)
    ham_predict = []
    for each_document in ham_email_bernoulli_model_test:
        ham_predict.append(test_mcap_logistic_regression(each_document, weights))
    ham_actual = [0] * len(ham_predict)
    total_actual = spam_actual + ham_actual
    total_predict = spam_predict + ham_predict
    # Now we find the evaluation metrics for the method
    accur = accuracy(total_actual, total_predict)
    prec = precision(total_actual, total_predict)
    rec = recall(total_actual, total_predict)
    f1 = f1_score(rec, prec)
    return accur, prec, rec, f1, lambda_parameter

In [206]:
evaluate_MCAP_bernoulli_model('/content/enron1')

(0.8947368421052632,
 0.9809523809523809,
 0.6912751677852349,
 0.8110236220472441,
 7)

In [207]:
evaluate_MCAP_bernoulli_model('/content/enron4')

(0.9613259668508287, 0.9490291262135923, 1.0, 0.9738480697384807, 1)

In [208]:
evaluate_MCAP_bernoulli_model('/content/hw1')

(0.9288702928870293,
 0.9705882352941176,
 0.7615384615384615,
 0.8534482758620688,
 5)

## SGDClassifier

In [209]:
def parameter_tuning(validation_x, validation_y):
    parameters_to_be_tuned = {'alpha': (0.01, 0.05),
                              'max_iter': (range(500, 3000, 1000)),
                              'learning_rate': ('optimal', 'invscaling', 'adaptive'),
                              'eta0': (0.3, 0.7),
                              'tol': (0.001, 0.005)
                              }
    SGDclassifier = SGDClassifier()
    gridSearch = GridSearchCV(SGDclassifier, parameters_to_be_tuned, cv=5)
    gridSearch.fit(validation_x, validation_y)
    return gridSearch

In [210]:
def train_SGD(train_x, train_y, classifier):
    return classifier.fit(train_x, train_y)

In [211]:
def test_SGD(trained_classifier, test_x, test_y):
    predicted_y = []
    for each_document in test_x:
        predicted_y.append(trained_classifier.predict(np.reshape(each_document, (1, -1))))
    return predicted_y, test_y

In [212]:
def convert_data_for_SGD_classifier(data, words_list):
    train_x = []
    train_y = []
    for each_document in data:
        train_x_for_this_document = []
        train_y.append(each_document["the_class_of_the_document"])
        for each_word in words_list:
            # use a try except here since it may happen that the given word is not in the document
            try:
                train_x_for_this_document.append(each_document[each_word])
            except:
                # If the word is not in the test set then we just 0 as the input for the given word.
                train_x_for_this_document.append(0)
        train_x.append(train_x_for_this_document)
    return train_x, train_y

In [213]:
def get_data_from_given_model(spam_email_model, ham_email_model):
    for ind in range(len(spam_email_model)):
        spam_email_model[ind]["the_class_of_the_document"] = 1
    for ind in range(len(ham_email_model)):
        ham_email_model[ind]["the_class_of_the_document"] = 0
      # combine spam email_model with ham_email_model
    all_data = spam_email_model + ham_email_model
    return all_data

In [214]:
def divide_into_validation_and_train(spam_email_model, ham_email_model):
    # Here spam is 1 and ham is 0 (since we are using sigmoid)
    for ind in range(len(spam_email_model)):
        spam_email_model[ind]["the_class_of_the_document"] = 1
    for ind in range(len(ham_email_model)):
        ham_email_model[ind]["the_class_of_the_document"] = 0
    all_data = spam_email_model + ham_email_model
    # We are using this step to shuffle our data so that different data goes into training and testing everything
    random.shuffle(all_data)
    train_data = all_data[0: int(len(all_data) * .70)]
    validation_data = all_data[int(len(all_data) * .70): -1]
    return train_data, validation_data

In [215]:
def evaluate_SGD_bag_of_words(path_of_dataset_name):
    # We first import training data for the training
    try:
        spam_email_bag_of_words, ham_email_bag_of_words, words_frequency_in_all_folders, words_frenquency_spam_email_all_folders, words_frenquency_ham_email_in_all_folders, num_of_files_whole_dataset, num_of_files_spam_dataset, num_of_files_ham_dataset, total_words_dictionary = bag_of_words_model(
            path_of_dataset_name, True)
        spam_email_bag_of_words_test, ham_email_bag_of_words_test, words_frequency_in_all_folders_test, words_frenquency_spam_email_all_folders_test, words_frenquency_ham_email_in_all_folders_test, num_of_files_whole_dataset_test, num_of_files_spam_dataset_test, num_of_files_ham_dataset_test, total_words_dictionary_test = bag_of_words_model(
            path_of_dataset_name, False)
    except:
        print( "wrong file")
        exit(-1)
    train_data, validation_data = divide_into_validation_and_train(spam_email_bag_of_words, ham_email_bag_of_words)
    test_data = get_data_from_given_model(spam_email_bag_of_words_test, ham_email_bag_of_words_test)
    words_list = list(train_data[0])
    # import the train, test and validation datasets
    train_x, train_y = convert_data_for_SGD_classifier(train_data, words_list)
    test_x, test_y = convert_data_for_SGD_classifier(test_data, words_list)
    valid_x, valid_y = convert_data_for_SGD_classifier(validation_data, words_list)
    # get the best parameters for the sklearn SGD classifier
    classifier_model = parameter_tuning(valid_x, valid_y)
    # In this step the classifier model is being trained on the training dataset
    trained_classifier_model = train_SGD(train_x, train_y, classifier_model)
    # In this step we find the output for the classifier.
    predicted_y, actual_y = test_SGD(trained_classifier_model, test_x, test_y)
    # Now calculate the evaluation metrics
    
    accur = accuracy(actual_y, predicted_y)
    prec = precision(actual_y, predicted_y)
    rec = recall(actual_y, predicted_y)
    f1 = f1_score(rec, prec)
    return accur, prec, rec, f1

In [216]:
def evaluate_SGD_bernoulli_model(path_of_dataset_name):
    # We first import training data for the training
    try:
        spam_email_bernoulli_model1, ham_email_bernoulli_model1, words_frenquency_spam_email_all_folders, words_frenquency_ham_email_in_all_folders, num_of_files_whole_dataset, num_of_files_spam_dataset, num_of_files_ham_dataset, total_words_dictionary = bernoulli_model(
            path_of_dataset_name, True)
        spam_email_bernoulli_model_test, ham_email_bernoulli_model_test, words_frenquency_spam_email_all_folders_test, words_frenquency_ham_email_in_all_folders_test, num_of_files_whole_dataset_test, num_of_files_spam_dataset_test, num_of_files_ham_dataset_test, total_words_dictionary_test = bernoulli_model(
            path_of_dataset_name, False)
    except:
        print ("wrong file")
        exit(-1)
    train_data, validation_data = divide_into_validation_and_train(spam_email_bernoulli_model1,
                                                                                 ham_email_bernoulli_model1)
    test_data = get_data_from_given_model(spam_email_bernoulli_model_test, ham_email_bernoulli_model_test)
    words_list = list(train_data[0])
    # import the train, test and validation datasets
    train_x, train_y = convert_data_for_SGD_classifier(train_data, words_list)
    test_x, test_y = convert_data_for_SGD_classifier(test_data, words_list)
    valid_x, valid_y = convert_data_for_SGD_classifier(validation_data, words_list)
    # get the best parameters for the sklearn SGD classifier
    classifier_model = parameter_tuning(valid_x, valid_y)
    # In this step the classifier model is being trained on the training dataset
    trained_classifier_model = train_SGD(train_x, train_y, classifier_model)
    # In this step we find the output for the classifier.
    predicted_y, actual_y = test_SGD(trained_classifier_model, test_x, test_y)
    # Now calculate the evaluation metrics
    accur = accuracy(actual_y, predicted_y)
    prec = precision(actual_y, predicted_y)
    rec = recall(actual_y, predicted_y)
    f1 = f1_score(rec, prec)
    return accur, prec, rec, f1

In [217]:
evaluate_SGD_bag_of_words('/content/enron1')


(0.9780701754385965, 0.9371069182389937, 1.0, 0.9675324675324676)

In [218]:
evaluate_SGD_bag_of_words('/content/enron4')

(0.9834254143646409,
 0.9798994974874372,
 0.9974424552429667,
 0.9885931558935361)

In [219]:
evaluate_SGD_bag_of_words('/content/hw1')

(0.9811715481171548,
 0.9481481481481482,
 0.9846153846153847,
 0.9660377358490566)

In [220]:
evaluate_SGD_bernoulli_model('/content/enron1')

(1.0, 1.0, 1.0, 1.0)

In [221]:
evaluate_SGD_bernoulli_model('/content/enron4')

(0.996316758747698, 0.9949109414758269, 1.0, 0.9974489795918366)

In [222]:
evaluate_SGD_bernoulli_model('/content/hw1')

(1.0, 1.0, 1.0, 1.0)